In [1]:
import pandas as pd
import os
from main_utils_1 import mk_dirs
import warnings
warnings.filterwarnings("ignore")

## Normalizing the virage and matbII with the respective baselines

In [3]:
# normalization of new features extracted from EDA with the new baseline features!
main_path = r"X:\Four modes baseline\Virage_Clipped_Baseline\Extracted\ECG_EDA_Features"

base_path = r"X:\RealTimeSegment\Driving Simulator\Extracted\ECG_EDA_baseline_oneline_std"
sess_base_path = r"X:\Four modes baseline\Virage_Clipped_Baseline\Extracted\ECG_EDA_baseline_oneline_std"

savePath_individual_feat = r"X:\Four modes baseline\Virage_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features"
label_path = r"X:\IDEaS_2\Driving Simulator\Data\New_Labels"
combined_feat = r"X:\Four modes baseline\Virage_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features_Combined"

subjects_id = os.listdir(main_path)

for sub_id in subjects_id:
    sub_path = os.path.join(main_path, sub_id)
    dirlist = os.listdir(sub_path)
    exp_id = [x for x in dirlist if '_featurs_' in x] # op: ['ecg_level_1.csv', 'ecg_level_2.csv', ...]
    exp_id = [x.replace('.csv', '') for x in exp_id]
    exp_id = [x.replace('ecg_featurs_', '') for x in exp_id]
    exp_id = [x.replace('eda_featurs_', '') for x in exp_id]
    exp_id = set(exp_id)

    sub_id = int(sub_id)
    print(sub_id)

    ''' read 3 minutes baseline for each sub '''
    ecg_base = os.path.join(base_path, str(sub_id), 'ecg_baseline_features_oneline.csv')
    eda_base = os.path.join(base_path, str(sub_id), 'eda_baseline_features_oneline.csv')
    df_baseline_ecg = pd.read_csv(ecg_base)
    df_baseline_eda = pd.read_csv(eda_base)

    ''' Add 'ecg_' to each column name for ECG features in 3min Baseline '''
    ecg_base_cols = list(df_baseline_ecg.columns)
    ecg_base_cols = [x.replace('ecg_', '') for x in ecg_base_cols]
    ecg_base_cols = ['ecg_' + x  for x in ecg_base_cols]

    df_baseline_ecg.columns = ecg_base_cols

    label_dir = os.path.join(label_path, '{}.csv'.format(sub_id))
    df_label = pd.read_csv(label_dir).add_prefix('label_')
    df_label['label_time'] = df_label['label_time'].apply(lambda x: x - 10)

    df_main = pd.DataFrame()
    for e_id in exp_id:
        try:
            eda_path = os.path.join(sub_path, 'eda_featurs_{}.csv'.format(e_id))
            ecg_path = os.path.join(sub_path, 'ecg_featurs_{}.csv'.format(e_id))

            df_ecg = pd.read_csv(ecg_path)
            df_eda = pd.read_csv(eda_path)

            df_ecg.drop(columns=['sess_id', 'subj_id'], inplace=True)
            df_eda.drop(columns=['sess_id', 'subj_id'], inplace=True)

            # break
            ecgTime = df_ecg['Timestamp'].copy()
            edaTime = df_eda['Timestamp'].copy()

            ''' Read the session based baseline for each subject and session '''
            sess_path = os.path.join(sess_base_path, str(sub_id))
            sess_ecg_baseline = os.path.join(sess_path, 'ecg_level{}_baseline_features_oneline.csv'.format(e_id))
            sess_eda_baseline = os.path.join(sess_path, 'eda_level{}_baseline_features_oneline.csv'.format(e_id))
            #### Replace the ecg_baseline and eda_baseline df if the session baseline exists!
            if os.path.exists(sess_ecg_baseline) and os.path.exists(sess_eda_baseline):
                df_baseline_ecg = pd.read_csv(sess_ecg_baseline)
                df_baseline_eda = pd.read_csv(sess_eda_baseline)

            df_ecg = df_ecg.sub(df_baseline_ecg.iloc[0])
            df_ecg = df_ecg.div(df_baseline_ecg.iloc[0])
            df_ecg['Timestamp'] = ecgTime

            df_eda = df_eda.sub(df_baseline_eda.iloc[0])
            df_eda = df_eda.div(df_baseline_eda.iloc[0])
            df_eda['Timestamp'] = edaTime

            df_label['label_com_level_1'] = [1] * len(df_label)
            df_label['label_com_level_2'] = [2] * len(df_label)
            df_label['label_com_level_3'] = [3] * len(df_label)
            df_label['label_com_level_4'] = [4] * len(df_label)
            df_label['label_com_level_5'] = [5] * len(df_label)
            df_label['label_com_level_6'] = [6] * len(df_label)
            df_label['label_com_level_7'] = [7] * len(df_label)
            df_label['label_com_level_8'] = [8] * len(df_label)
            df_label['label_com_level_9'] = [9] * len(df_label)
            
            # Extracting releveant label columns for each experiment
            label_exp = ['label_', 'label_com_']
            relevant_label_columns = [x + 'level_' + e_id for x in label_exp]
            relevant_label_columns.append('label_time')
            df_rel_label = df_label[relevant_label_columns].copy()
            df_rel_label.columns = ['label', 'complexity', 'label_time']

            df_ecg.index = df_ecg['Timestamp'].values
            df_eda.index = df_eda['Timestamp'].values
            # df_label = df_ecg['label'].copy()

            df_ecg.drop(columns=['Timestamp'], inplace=True) # removing Timestamps columns from one of the DataFrames

            merged = pd.concat([df_ecg, df_eda], axis = 1)
            df_merged = pd.merge(merged, df_rel_label, left_on='Timestamp', right_on='label_time', how='left')
            df_merged.drop(columns=['Timestamp', 'label_time'], inplace=True)

            df_main = df_main.append(df_merged)
            df_main = df_main.reset_index(drop=True)

            indv_path = os.path.join(savePath_individual_feat, str(sub_id))
            mk_dirs(indv_path)

            df_merged.to_csv(os.path.join(indv_path, '{}.csv'.format(e_id)),
                            index = False)
        except FileNotFoundError:
            continue
    
    mk_dirs(combined_feat)
    df_main.dropna(subset=['label'], inplace=True)
    df_main.to_csv(os.path.join(combined_feat, '{}.csv'.format(sub_id)), index = False)

1030
1105
1106
1241
1271
1314
1323
1337
1372
1417
1434
1544
1547
1595
1629
1716
1717
1744
1868
1892
1953


In [4]:
def normalize_labels(values, actual_bounds, desired_bounds=(1, 9)):
    return [desired_bounds[0] + (x - actual_bounds[0]) * (desired_bounds[1] - desired_bounds[0]) / (actual_bounds[1] - actual_bounds[0]) for x in values]

In [8]:
# add Scaled labels in the csv files for each subject

subFile = r"X:\Four modes baseline\Virage_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features_Combined"
subScldFile = r"X:\Four modes baseline\Virage_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features_Combined_scld"

desMin = 1
desMax = 9

subjects = os.listdir(subFile)

subjects = [x.replace('.csv', '') for x in subjects]
for subs in subjects:
    try:
        subPath = os.path.join(subFile, '{}.csv'.format(subs))

        subDf = pd.read_csv(subPath)
        subMax = subDf['label'].max()
        subMin = subDf['label'].min()

        scLab = normalize_labels(subDf['label'].values, (subMin, subMax), (desMin, desMax))
        scLab = [round(x) for x in scLab]

        subDf['scaled label'] = scLab
        mk_dirs(subScldFile)
        subDf.to_csv(os.path.join(subScldFile, '{}.csv'.format(subs)), index = False)
    except FileNotFoundError:
        continue

Didn't rung the below code # resused the previous features

In [7]:
# main_path = r"X:\RealTimeSegment\Driving Simulator\Extracted\ECG_EDA_baseline_Features"
# base_path = r"X:\RealTimeSegment\Driving Simulator\Extracted\ECG_EDA_baseline_oneline_std"
# # base_path = r"X:\RealTimeSegment\Driving Simulator\Extracted\ECG_EDA_baseline_oneline"

# savePath_individual_feat = r"X:\RealTimeSegment\Driving Simulator\Extracted\Norm_ECG_EDA_Features_Baseline"
# label_path = r"X:\IDEaS_2\Driving Simulator\Data\New_Labels"
# combined_feat = r"X:\RealTimeSegment\Driving Simulator\Extracted\Norm_ECG_EDA_Features_Baseline_Combined"

# subjects_id = os.listdir(main_path)

# for sub_id in subjects_id:
#     sub_path = os.path.join(main_path, sub_id)
#     dirlist = os.listdir(sub_path)
#     # exp_id = [x for x in dirlist if '_featurs_' in x] # op: ['ecg_level_1.csv', 'ecg_level_2.csv', ...]
#     # exp_id = [x.replace('.csv', '') for x in exp_id]
#     # exp_id = [x.replace('ecg_baseline_', '') for x in exp_id]
#     # exp_id = [x.replace('eda_baseline_', '') for x in exp_id]
#     # exp_id = set(exp_id)
#     exp_id = ['baseline']
#     sub_id = int(sub_id)

#     ecg_base = os.path.join(base_path, str(sub_id), 'ecg_baseline_features_oneline.csv')
#     eda_base = os.path.join(base_path, str(sub_id), 'eda_baseline_features_oneline.csv')

#     df_baseline_ecg = pd.read_csv(ecg_base)
#     df_baseline_eda = pd.read_csv(eda_base)

#     ecg_base_cols = list(df_baseline_ecg.columns)
#     ecg_base_cols = [x.replace('ecg_', '') for x in ecg_base_cols]
#     ecg_base_cols = ['ecg_' + x  for x in ecg_base_cols]

#     df_baseline_ecg.columns = ecg_base_cols
#     df_main = pd.DataFrame()
#     # break

#     for e_id in exp_id:
#         try:
#             eda_path = os.path.join(sub_path, 'eda_{}_featurs.csv'.format(e_id))
#             ecg_path = os.path.join(sub_path, 'ecg_{}_featurs.csv'.format(e_id))

#             df_ecg = pd.read_csv(ecg_path)
#             df_eda = pd.read_csv(eda_path)

#             # break
#             df_ecg.drop(columns=['subj_id'], inplace=True)
#             df_eda.drop(columns=['subj_id'], inplace=True)

#             # break
#             ecgTime = df_ecg['Timestamp'].copy()
#             edaTime = df_eda['Timestamp'].copy()

#             # df_ecg = df_ecg.sub(df_baseline_ecg.iloc[0])
#             df_ecg = df_ecg.div(df_baseline_ecg.iloc[0])
#             df_ecg['Timestamp'] = ecgTime

#             # df_eda = df_eda.sub(df_baseline_eda.iloc[0])
#             df_eda = df_eda.div(df_baseline_eda.iloc[0])
#             df_eda['Timestamp'] = edaTime
            
#             df_ecg.index = df_ecg['Timestamp'].values
#             df_eda.index = df_eda['Timestamp'].values
#             # df_label = df_ecg['label'].copy()

#             df_ecg.drop(columns=['Timestamp'], inplace=True) # removing Timestamps columns from one of the DataFrames

#             merged = pd.concat([df_ecg, df_eda], axis = 1)
#             merged['label'] = [0] * len(merged)
#             merged['complexity'] = [0] * len(merged)

#             # df_merged = pd.merge(merged, df_rel_label, left_on='Timestamp', right_on='label_time', how='left')
#             merged.drop(columns=['Timestamp'], inplace=True)

#             df_main = df_main.append(merged)
#             df_main = df_main.reset_index(drop=True)

#             # break
#             indv_path = os.path.join(savePath_individual_feat, str(sub_id))
#             mk_dirs(indv_path)

#             merged.to_csv(os.path.join(indv_path, '{}.csv'.format(e_id)),
#                             index = False)
#         except FileNotFoundError as e:
#             print(e)
#             continue
#     # break
#     mk_dirs(combined_feat)
#     df_main.dropna(subset=['label'], inplace=True)
#     df_main.to_csv(os.path.join(combined_feat, '{}.csv'.format(sub_id)), index = False)

## MatBII Normalization

In [7]:
main_path = r"X:\Four modes baseline\MatB-II_Clipped_Baseline\Extracted\ECG_EDA_Features"
base_path = r"X:\RealTimeSegment\MatbII\Extracted\ECG_EDA_baseline_oneline_std"
sess_base_path = r"X:\Four modes baseline\MatB-II_Clipped_Baseline\Extracted\ECG_EDA_baseline_oneline_std"

# base_path = r"X:\RealTimeSegment\MatbII\Extracted\ECG_EDA_baseline_oneline"


savePath_individual_feat = r"X:\Four modes baseline\MatB-II_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features"
label_path = r"X:\IDEaS_2\MATBII\Data\New_Labels_2"
combined_feat = r"X:\Four modes baseline\MatB-II_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features_Combined"

subjects_id = os.listdir(main_path)

for sub_id in subjects_id:
    sub_path = os.path.join(main_path, sub_id)
    dirlist = os.listdir(sub_path)
    exp_id = [x for x in dirlist if '_featurs_' in x] # op: ['ecg_level_1.csv', 'ecg_level_2.csv', ...]
    exp_id = [x.replace('.csv', '') for x in exp_id]
    exp_id = [x.replace('ecg_featurs_', '') for x in exp_id]
    exp_id = [x.replace('eda_featurs_', '') for x in exp_id]
    exp_id = set(exp_id)

    sub_id = int(sub_id)
    print(sub_id)

    ecg_base = os.path.join(base_path, str(sub_id), 'ecg_baseline_features_oneline.csv')
    eda_base = os.path.join(base_path, str(sub_id), 'eda_baseline_features_oneline.csv')

    df_baseline_ecg = pd.read_csv(ecg_base)
    df_baseline_eda = pd.read_csv(eda_base)

    ecg_base_cols = list(df_baseline_ecg.columns)
    ecg_base_cols = [x.replace('ecg_', '') for x in ecg_base_cols]
    ecg_base_cols = ['ecg_' + x  for x in ecg_base_cols]

    df_baseline_ecg.columns = ecg_base_cols

    label_dir = os.path.join(label_path, '{}.csv'.format(sub_id))
    df_label = pd.read_csv(label_dir).add_prefix('label_')
    df_label['label_time'] = df_label['label_time'].apply(lambda x: x - 10)

    # break
    df_main = pd.DataFrame()
    for e_id in exp_id:
        try:
            eda_path = os.path.join(sub_path, 'eda_featurs_{}.csv'.format(e_id))
            ecg_path = os.path.join(sub_path, 'ecg_featurs_{}.csv'.format(e_id))

            df_ecg = pd.read_csv(ecg_path)
            df_eda = pd.read_csv(eda_path)

            df_ecg.drop(columns=['sess_id', 'subj_id'], inplace=True)
            df_eda.drop(columns=['sess_id', 'subj_id'], inplace=True)

            ecgTime = df_ecg['Timestamp'].copy()
            edaTime = df_eda['Timestamp'].copy()

            ''' Read the session based baseline for each subject and session '''
            sess_path = os.path.join(sess_base_path, str(sub_id))
            sess_ecg_baseline = os.path.join(sess_path, 'ecg_exp{}_baseline_features_oneline.csv'.format(e_id))
            sess_eda_baseline = os.path.join(sess_path, 'eda_exp{}_baseline_features_oneline.csv'.format(e_id))
            #### Replace the ecg_baseline and eda_baseline df if the session baseline exists!
            if os.path.exists(sess_ecg_baseline) and os.path.exists(sess_eda_baseline):
                print('session baseline exists')
                df_baseline_ecg = pd.read_csv(sess_ecg_baseline)
                df_baseline_eda = pd.read_csv(sess_eda_baseline)            

            df_ecg = df_ecg.sub(df_baseline_ecg.iloc[0])
            df_ecg = df_ecg.div(df_baseline_ecg.iloc[0])
            df_ecg['Timestamp'] = ecgTime

            df_eda = df_eda.sub(df_baseline_eda.iloc[0])
            df_eda = df_eda.div(df_baseline_eda.iloc[0])
            df_eda['Timestamp'] = edaTime

            # Extracting releveant label columns for each experiment
            label_exp = ['label_', 'label_com_']
            relevant_label_columns = [x + 'exp_' + e_id for x in label_exp]
            relevant_label_columns.append('label_time')
            df_rel_label = df_label[relevant_label_columns].copy()
            df_rel_label.columns = ['label', 'complexity', 'label_time']

            df_ecg.index = df_ecg['Timestamp'].values
            df_eda.index = df_eda['Timestamp'].values
            # df_label = df_ecg['label'].copy()

            df_ecg.drop(columns=['Timestamp'], inplace=True) # removing Timestamps columns from one of the DataFrames

            merged = pd.concat([df_ecg, df_eda], axis = 1)
            df_merged = pd.merge(merged, df_rel_label, left_on='Timestamp', right_on='label_time', how='left')
            df_merged.drop(columns=['Timestamp', 'label_time'], inplace=True)

            df_main = df_main.append(df_merged)
            df_main = df_main.reset_index(drop=True)

            indv_path = os.path.join(savePath_individual_feat, str(sub_id))
            mk_dirs(indv_path)

            # break
            df_merged.to_csv(os.path.join(indv_path, '{}.csv'.format(e_id)),
                            index = False)
        except FileNotFoundError:
            continue
    # break
    mk_dirs(combined_feat)
    df_main.dropna(subset=['label'], inplace=True)
    df_main.to_csv(os.path.join(combined_feat, '{}.csv'.format(sub_id)), index = False)

1026
session baseline exists
session baseline exists
session baseline exists
session baseline exists
1105
session baseline exists
session baseline exists
session baseline exists
session baseline exists
1106
session baseline exists
session baseline exists
session baseline exists
session baseline exists
1175
session baseline exists
1194
session baseline exists
session baseline exists
session baseline exists
1337
session baseline exists
session baseline exists
session baseline exists
session baseline exists
1390
session baseline exists
session baseline exists
session baseline exists
session baseline exists
1400
1419
session baseline exists
session baseline exists
session baseline exists
1517
1544
session baseline exists
1624
1629
session baseline exists
session baseline exists
1674
session baseline exists
session baseline exists
session baseline exists
1688
1717
session baseline exists
session baseline exists
session baseline exists
1765
session baseline exists
session baseline exists
ses

In [9]:
# add Scaled labels in the csv files for each subject

subFile = r"X:\Four modes baseline\MatB-II_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features_Combined"
subScldFile = r"X:\Four modes baseline\MatB-II_Clipped_Baseline\Extracted\Norm_ECG_EDA_Features_Combined_scld"

desMin = 1
desMax = 9

subjects = os.listdir(subFile)

subjects = [x.replace('.csv', '') for x in subjects]
for subs in subjects:
    try:
        subPath = os.path.join(subFile, '{}.csv'.format(subs))

        subDf = pd.read_csv(subPath)
        subMax = subDf['label'].max()
        subMin = subDf['label'].min()

        scLab = normalize_labels(subDf['label'].values, (subMin, subMax), (desMin, desMax))
        scLab = [round(x) for x in scLab]

        subDf['scaled label'] = scLab
        mk_dirs(subScldFile)
        subDf.to_csv(os.path.join(subScldFile, '{}.csv'.format(subs)), index = False)
    except FileNotFoundError:
        continue